In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [14]:
file = "/home/duo/proyecto-computacion-I/data/datasets/raw/Censo vertidos.xlsx"

df = pd.read_excel(file, header=4)

print("Primeras 5 filas:")
df.head()

print("\nInformación del DataFrame:")
df.info()

Primeras 5 filas:

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1837 entries, 0 to 1836
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              0 non-null      float64
 1   NOMBRE DEL VERTIDO      1836 non-null   object 
 2   TITULAR                 1835 non-null   object 
 3   MUNICIPIO DEL VERTIDO   1835 non-null   object 
 4   PROVINCIA VERTIDO       1835 non-null   object 
 5   UTM X 
Huso 30          1835 non-null   float64
 6   UTM Y 
Huso 30          1835 non-null   float64
 7   MEDIO     RECEPTOR      1835 non-null   object 
 8   NATURALEZA DEL VERTIDO  1835 non-null   object 
 9   CARACT. DEL VERTIDO     1835 non-null   object 
 10  VOLUMEN (m3/año)        1835 non-null   float64
 11  NATURALEZA M.RECEPTOR   1835 non-null   object 
 12  REF. EXPED.             1835 non-null   object 
dtypes: float64(4), object(9)
memory usage: 186.7+ K

# Limpieza

In [15]:
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

Limpiar y renombrar columnas para que sean más fáciles de usar (Quitar \n, espacios y paréntesis)

In [16]:
df.columns = df.columns.str.replace('\n', ' ', regex=False).str.strip()

df = df.rename(columns={
    'NOMBRE DEL VERTIDO': 'NOMBRE_VERTIDO',
    'MUNICIPIO DEL VERTIDO': 'MUNICIPIO',
    'PROVINCIA VERTIDO': 'PROVINCIA',
    'UTM X Huso 30': 'UTM_X',
    'UTM Y Huso 30': 'UTM_Y',
    'MEDIO RECEPTOR': 'MEDIO_RECEPTOR',
    'NATURALEZA DEL VERTIDO': 'NATURALEZA_VERTIDO',
    'CARACT. DEL VERTIDO': 'CARACT_VERTIDO',
    'VOLUMEN (m3/año)': 'VOLUMEN_m3_ano',
    'NATURALEZA M.RECEPTOR': 'NATURALEZA_RECEPTOR',
    'REF. EXPED.': 'REF_EXPEDIENTE'
})

Eliminar las filas que tienen valores nulos en columnas clave (ej. TITULAR o MUNICIPIO) Esto eliminará esas 2 filas que parecían vacías

In [17]:
df = df.dropna(subset=['TITULAR', 'MUNICIPIO', 'VOLUMEN_m3_ano'])

In [18]:
print("Datos limpios (primeras 5 filas):")
df.head()

Datos limpios (primeras 5 filas):


,NOMBRE_VERTIDO,TITULAR,MUNICIPIO,PROVINCIA,UTM X Huso 30,UTM Y Huso 30,MEDIO RECEPTOR,NATURALEZA_VERTIDO,CARACT_VERTIDO,VOLUMEN_m3_ano,NATURALEZA_RECEPTOR,REF_EXPEDIENTE
0,DROMEN GESTORES Y CONSULTORES (VIV. UNIFAMILIAR),"DROMEN GESTORES Y CONSULTORES, S.L.",ADRADA (LA),AVILA,358144.0,4460601.0,TERRENO,URBANO O ASIMILABLE,<2.000 h-e,231.0,CATEGORÍA I,AV-0006/2020
1,PIZZERIA-RESTAURANTE PUENTE MOCHA,PERSONA FÍSICA,ADRADA (LA),AVILA,358290.0,4459633.0,TERRENO,URBANO O ASIMILABLE,<2.000 h-e,1188.0,CATEGORÍA I,AV-0049/1999
2,CENTRO HÍPICO LA ESPUELA,"CORPORACIÓN RILUMI, S.L.",ADRADA (LA),AVILA,358288.0,4459174.0,TERRENO,URBANO O ASIMILABLE,<2.000 h-e,3650.0,CATEGORÍA I,AV-0041/2015
3,EDAR LA ADRADA,AYUNTAMIENTO DE LA ADRADA,ADRADA (LA),AVILA,359461.0,4460939.0,ARROYO FRANQUILLO,URBANO O ASIMILABLE,2.000-10.000 h-e,1193058.0,CATEGORÍA I,AV-0053/2014
4,PERSONA FÍSICA (VIVIENDAS),PERSONA FÍSICA,ADRADA (LA),AVILA,357353.0,4461118.0,TERRENO,URBANO O ASIMILABLE,<2.000 h-e,102.0,CATEGORÍA I,AV-0041/2017


In [19]:
print("\nNueva información del DataFrame:")
df.info()


Nueva información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 1835 entries, 0 to 1834
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NOMBRE_VERTIDO       1835 non-null   object 
 1   TITULAR              1835 non-null   object 
 2   MUNICIPIO            1835 non-null   object 
 3   PROVINCIA            1835 non-null   object 
 4   UTM X  Huso 30       1835 non-null   float64
 5   UTM Y  Huso 30       1835 non-null   float64
 6   MEDIO     RECEPTOR   1835 non-null   object 
 7   NATURALEZA_VERTIDO   1835 non-null   object 
 8   CARACT_VERTIDO       1835 non-null   object 
 9   VOLUMEN_m3_ano       1835 non-null   float64
 10  NATURALEZA_RECEPTOR  1835 non-null   object 
 11  REF_EXPEDIENTE       1835 non-null   object 
dtypes: float64(3), object(9)
memory usage: 186.4+ KB


Clasificar los titulares

In [27]:
def clasificar_titular(titular):
    """
    Clasifica el titular basándose en ciertas palabras clave.
    Retorna: 'A', 'F', 'J', o np.nan si no se clasifica.
    """
    if pd.isna(titular):
        return np.nan # Manejar valores nulos si existen

    # Convertir a mayúsculas para asegurar una búsqueda sin distinción
    titular_upper = str(titular).upper()

    # Clasificación 'A' - AYUNTAMIENTO
    terminos_admin = ['AYUNTAMIENTO', 'MINISTERIO', 'JUNTA', 'ACUARTELAMIENTO']
    if any(term in titular_upper for term in terminos_admin):
        return 'A'

    # Clasificación 'F' - PERSONA FÍSICA
    # Buscamos 'PERSONA FÍSICA' o sinónimos comunes de 'particular'
    if 'PERSONA FÍSICA' in titular_upper or 'PARTICULAR' in titular_upper:
        return 'F'

    # Clasificación 'J' - PERSONAS JURÍDICAS (Sociedades)
    # Lista de términos comunes para personas jurídicas
    terminos_juridicos = ['S.L.', 'SL.', 'S.L', 'S. A.', 'SA.', 'S.A', 'SOCIEDAD', 'CORPORACION', 'FUNDACIÓN', 'SLU', 'SAU', 'SRL', 'COMUNIDAD', 'COM', 'ASOCIACIÓN', 'PARROQUIA']

    # Verificamos si alguno de los términos jurídicos está en el titular
    if any(term in titular_upper for term in terminos_juridicos):
        return 'J'

    # Si no coincide con ninguna de las categorías anteriores
    return np.nan # Usamos np.nan (Not a Number) para indicar no clasificado

In [28]:
df['tipo_titular'] = df['TITULAR'].apply(clasificar_titular)

In [29]:
df[df['tipo_titular'].isnull()]

,NOMBRE_VERTIDO,TITULAR,MUNICIPIO,PROVINCIA,UTM X Huso 30,UTM Y Huso 30,MEDIO RECEPTOR,NATURALEZA_VERTIDO,CARACT_VERTIDO,VOLUMEN_m3_ano,NATURALEZA_RECEPTOR,REF_EXPEDIENTE,tipo_titular
13,RESTAURANTE GRUTAS DEL AGUILA,"HERMANOS MUÑOZ PEREDA, C.B.",ARENAS DE SAN PEDRO,AVILA,323675.0,4447222.0,TERRENO,URBANO O ASIMILABLE,<2.000 h-e,2600.0,CATEGORÍA I,AV-0050/1999,NaN
33,EATON PLANNING LIMITED (VIV. UNIFAMILIAR),EATON PLANNING LIMITED,ARENAS DE SAN PEDRO,AVILA,317131.0,4441014.0,TERRENO,URBANO O ASIMILABLE,<2.000 h-e,438.0,CATEGORÍA I,AV-0111/2011,NaN
59,CAMPAMENTO SANTA MARÍA DEL MONTE,CONGREGACIÓN LEGIONARIOS DE CRISTO,BURGOHONDO,AVILA,348368.0,4473405.0,TERRENO,URBANO O ASIMILABLE,<2.000 h-e,730.0,CATEGORÍA I,AV-0020/2019,NaN
119,HOSTELERÍA EL CORRALÓN,GREDOS SAN DIEGO S. COOP. MAD.,CASAVIEJA,AVILA,348843.0,4462227.0,TERRENO,URBANO O ASIMILABLE,<2.000 h-e,6000.0,CATEGORÍA I,AV-0072/2021,NaN
178,COLONIA INFANTIL VENERO CLARO,CAJA DE AHORROS Y M. DE P. DE AVILA,NAVALUENGA,AVILA,358772.0,4474592.0,GARGANTA CABRERA,URBANO O ASIMILABLE,<2.000 h-e,2000.0,CATEGORÍA I,AV-0027/1994,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1819,URB. VEGA DEL CASTILLO,E.U.C.C. VEGA DEL CASTILLO,VISO DE SAN JUAN (EL),TOLEDO,415310.0,4443982.0,RÍO GUADARRAMA,URBANO O ASIMILABLE,<2.000 h-e,7300.0,CATEGORÍA I,AV-0050/2009,NaN
1820,URB. LAS ZANJAS,E.U.C.C. LAS ZANJAS,VISO DE SAN JUAN (EL),TOLEDO,419571.0,4444971.0,AFLUENTE DEL ARROYO VALSECO,URBANO O ASIMILABLE,<2.000 h-e,11680.0,CATEGORÍA I,AV-0104/2008,NaN
1822,URB. CAMINO DE LOS FRAILES,E.U.C.C. CAMINO DE LOS FRAILES,VISO DE SAN JUAN (EL),TOLEDO,414490.0,4444673.0,ARROYO PRADO DE LOS CABALLOS,URBANO O ASIMILABLE,<2.000 h-e,16425.0,CATEGORÍA I,AV-0090/2001,NaN
1823,URB. SAN MARCOS,E.U.C.C. URB. SAN MARCOS,VISO DE SAN JUAN (EL),TOLEDO,415794.0,4444407.0,RÍO GUADARRAMA,URBANO O ASIMILABLE,<2.000 h-e,34310.0,CATEGORÍA I,AV-0605/1987,NaN
